# Neo4j에 그래프 로드

In [22]:
import os
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"

In [23]:
import pickle

with open('samples/graph_docs.pkl', 'rb') as f:
    loaded_graph_docs = pickle.load(f)

In [24]:
from langchain.graphs import Neo4jGraph
from py2neo import Graph

py_graph = Graph()
n4j_graph = Neo4jGraph()
vector_search_enabled = True
index_name = "content_embedding_index"

/home/ubuntu/.local/lib/python3.10/site-packages/langchain_community/graphs/neo4j_graph.py:370: ExperimentalWarning: The configuration may change in the future.
  self._driver.verify_connectivity()


In [25]:
def init_graph(vector_search_enabled: bool):
    py_graph.run("MATCH (n) DETACH DELETE n")

    if vector_search_enabled:
        query = """
            CREATE VECTOR INDEX $index_name IF NOT EXISTS
            FOR (n:Content) ON (n.embedding)
            OPTIONS {
                indexConfig: {
                    `vector.dimensions`: 1024,
                    `vector.similarity_function`: 'cosine'
                }
            }
        """
        params = {"index_name": index_name}
        py_graph.run(query, params)

    py_graph.run("""
        CREATE FULLTEXT INDEX Search_Content_by_FullText IF NOT EXISTS
        FOR (n:Content) 
        ON EACH [n.text] 
        OPTIONS { indexConfig: { `fulltext.analyzer`: "english"}}
        """)

init_graph(vector_search_enabled)

n4j_graph.add_graph_documents(
  loaded_graph_docs, 
  baseEntityLabel=True
)

### Vector Search (Global)

In [54]:
from langchain_aws import BedrockEmbeddings
from langchain.vectorstores import Neo4jVector

if vector_search_enabled == True:
    embeddings = BedrockEmbeddings(model_id="cohere.embed-multilingual-v3", region_name="us-east-1")

    vector_store = Neo4jVector.from_existing_index(
        embedding=embeddings,
        index_name=index_name,
        node_label="Content",
        text_node_property="text", 
        embedding_node_property="embedding"
    )

    question = "Bedrock의 Agent에서 API 스키마를 설정하는 방법"    
    question_embedding = embeddings.embed_query(question)
    k=5

    cypher_query = f"""
        CALL db.index.vector.queryNodes('{index_name}', $k, $question_embedding) YIELD node, score
        WITH DISTINCT node, score
        WHERE node:Content
        RETURN node.text AS text, score
        ORDER BY score DESC
    """

    params = {
        "question_embedding": question_embedding,
        "k": k
    }

    results = vector_store.query(cypher_query, params=params)

    for result in results:
        print("=================\n")
        print(result['text'])
        print("=================\n")


learn how to define an action group with different methods. Topics - Define function details for your agent's action groups in Amazon Bedrock - Define OpenAPI schemas for your agent's action groups in Amazon Bedrock


SDK or CLI - Scenarios for Agents for Amazon Bedrock using AWS SDKs - An end-to-end example showing how to create and invoke Amazon Bedrock agents using an AWS SDK - Build and orchestrate generative AI applications with Amazon Bedrock and Step Functions


In the Action group type section, select one of the following methods for defining the parameters that the agent can elicit from users to help carry out actions: a. Define with function details – Define parameters for your agent to elicit from the user in order to carry out the actions. For more information on adding functions, see Define function details for your agent's action groups in Amazon Bedrock. b. Define with API schemas – Define the API operations that the agent can invoke and the parameters . Use an OpenAPI 

### Vector Search (Node Level)

In [47]:
if vector_search_enabled == True:

    vector_store = Neo4jVector.from_existing_index(
        embedding=embeddings,
        index_name=index_name,
        node_label="Content",
        text_node_property="text", 
        embedding_node_property="embedding"
    )
    
    find_parent_query = """
    MATCH (parent:Title)
    WHERE parent.value = $parent_value
    RETURN id(parent) AS parent_id
    """

    parent_params = {
        "parent_value": "Define OpenAPI schemas for your agent's action groups in Amazon Bedrock"
    }

    parent_result = vector_store.query(find_parent_query, params=parent_params)
    parent_id = parent_result[0]['parent_id'] if parent_result else None

    print("parent_id:", parent_id)
    if parent_id is None:
        print("Parent node not found")
    else:
        embeddings = BedrockEmbeddings(model_id="cohere.embed-multilingual-v3", region_name="us-east-1")

        question = "Bedrock의 Agent에서 API 스키마를 설정하는 방법"
        question_embedding = embeddings.embed_query(question)
        k=5
        cypher_query = """
            MATCH (parent)-[:HAS_CONTENTS]->(child:Content)
            WHERE id(parent) = $parent_id
            WITH child
            CALL db.index.vector.queryNodes($index_name, $k, $question_embedding) YIELD node, score
            WHERE node = child
            RETURN id(node) AS node_id, node.text AS text, score, {} AS metadata
            ORDER BY score DESC
            LIMIT $k
        """

        params = {
            "parent_id": parent_id,
            "question_embedding": question_embedding,
            "k": k,
            "index_name": index_name
        }

        results = vector_store.query(cypher_query, params=params)

        for result in results:
            print("=================\n")
            print(f"Node ID: {result['node_id']}")
            print(f"Text: {result['text']}")
            print("=================\n")



parent_id: 1404

Node ID: 1405
Text: When you create an action group in Amazon Bedrock, you must define the parameters that the agent needs to invoke from the user. You can also define API operations that the agent can invoke using these parameters. To define the API operations, create an OpenAPI schema in JSON or YAML format. You can create OpenAPI schema files and upload them to Amazon Simple Storage Service (Amazon S3). Alternatively, you can use the OpenAPI text editor in the console, which will validate your schema. After you create an agent, you can use the text editor when you add an action group to the agent or edit an existing action group. For more information, see Edit an agent. The agent uses the schema to determine the API operation that it needs to invoke and the parameters that are required to make the API request. These details are then sent through a Lambda function that you define to carry out the action or returned in the response of the agent invocation. Defining ac

### Vector Search (SubGraph Level)

In [53]:
if vector_search_enabled == True:

    vector_store = Neo4jVector.from_existing_index(
        embedding=embeddings,
        index_name=index_name,
        node_label="Content",
        text_node_property="text", 
        embedding_node_property="embedding"
    )
    
    embeddings = BedrockEmbeddings(model_id="cohere.embed-multilingual-v3", region_name="us-east-1")

    question = "Bedrock의 Agent에서 API 스키마를 설정하는 방법"
    question_embedding = embeddings.embed_query(question)
    k=5

    subgraph = "Amazon Bedrock"
    cypher_query = """
        MATCH (root:Title {level: "1", value: $subgraph})
        MATCH (root)-[:HAS_CHILD*0..]->(title:Title)-[:HAS_CONTENTS]->(content:Content)

        CALL db.index.vector.queryNodes($index_name, $k, $question_embedding) YIELD node, score
        WHERE node = content

        RETURN node.text AS text, score
        ORDER BY score DESC
        LIMIT $k
    """

    params = {
        "subgraph": subgraph,
        "question_embedding": question_embedding,
        "k": k,
        "index_name": index_name
    }

    results = vector_store.query(cypher_query, params=params)

    for result in results:
        print("=================\n")
        print(f"Text: {result['text']}")
        print("=================\n")




Text: learn how to define an action group with different methods. Topics - Define function details for your agent's action groups in Amazon Bedrock - Define OpenAPI schemas for your agent's action groups in Amazon Bedrock


Text: SDK or CLI - Scenarios for Agents for Amazon Bedrock using AWS SDKs - An end-to-end example showing how to create and invoke Amazon Bedrock agents using an AWS SDK - Build and orchestrate generative AI applications with Amazon Bedrock and Step Functions


Text: In the Action group type section, select one of the following methods for defining the parameters that the agent can elicit from users to help carry out actions: a. Define with function details – Define parameters for your agent to elicit from the user in order to carry out the actions. For more information on adding functions, see Define function details for your agent's action groups in Amazon Bedrock. b. Define with API schemas – Define the API operations that the agent can invoke and the parameters